In [12]:
from sklearn.datasets import load_breast_cancer
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
import seaborn as sns

url = "https://raw.githubusercontent.com/mwaskom/seaborn-data/master/diamonds.csv" 
df = pd.read_csv(url) 

In [13]:
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras

df

,carat,cut,color,clarity,depth,table,price,x,y,z
0,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
1,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31
2,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31
3,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63
4,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75
...,...,...,...,...,...,...,...,...,...,...
53935,0.72,Ideal,D,SI1,60.8,57.0,2757,5.75,5.76,3.50
53936,0.72,Good,D,SI1,63.1,55.0,2757,5.69,5.75,3.61
53937,0.70,Very Good,D,SI1,62.8,60.0,2757,5.66,5.68,3.56
53938,0.86,Premium,H,SI2,61.0,58.0,2757,6.15,6.12,3.74


In [14]:
# 컬렴명 확인
df.columns

Index(['carat', 'cut', 'color', 'clarity', 'depth', 'table', 'price', 'x', 'y',
       'z'],
      dtype='object')

In [15]:
y=df['price']
y.value_counts()

price
605      132
802      127
625      126
828      125
776      124
        ... 
8816       1
14704      1
14699      1
14698      1
9793       1
Name: count, Length: 11602, dtype: int64

In [16]:
# X_encoded = pd.get_dummies(X, columns=['cut','color','clarity'], drop_first=True)
# 0,1 로 표현하는 원핫인코딩
# 원핫 인코딩 (Sex 열만)

df_cut_encoded = pd.get_dummies(df['cut'], prefix='cut')

# 숫자형으로 변환
df_cut_encoded = df_cut_encoded.astype(int)

# 원래 데이터프레임과 원핫 인코딩된 데이터프레임 합치기
df = pd.concat([df.drop('cut', axis=1), df_cut_encoded], axis=1)

df_color_encoded = pd.get_dummies(df['color'], prefix='color')

# 숫자형으로 변환
df_color_encoded = df_color_encoded.astype(int)

# 원래 데이터프레임과 원핫 인코딩된 데이터프레임 합치기
df = pd.concat([df.drop('color', axis=1), df_color_encoded], axis=1)

df_clarity_encoded = pd.get_dummies(df['clarity'], prefix='cut')

# 숫자형으로 변환
df_clarity_encoded = df_clarity_encoded.astype(int)

# 원래 데이터프레임과 원핫 인코딩된 데이터프레임 합치기
df = pd.concat([df.drop('clarity', axis=1), df_clarity_encoded], axis=1)

In [17]:
X=df.drop('price',axis=1)
X.head()

,carat,depth,table,x,y,z,cut_Fair,cut_Good,cut_Ideal,cut_Premium,...,color_I,color_J,cut_I1,cut_IF,cut_SI1,cut_SI2,cut_VS1,cut_VS2,cut_VVS1,cut_VVS2
0,0.23,61.5,55.0,3.95,3.98,2.43,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
1,0.21,59.8,61.0,3.89,3.84,2.31,0,0,0,1,...,0,0,0,0,1,0,0,0,0,0
2,0.23,56.9,65.0,4.05,4.07,2.31,0,1,0,0,...,0,0,0,0,0,0,1,0,0,0
3,0.29,62.4,58.0,4.20,4.23,2.63,0,0,0,1,...,1,0,0,0,0,0,0,1,0,0
4,0.31,63.3,58.0,4.34,4.35,2.75,0,1,0,0,...,0,1,0,0,0,1,0,0,0,0


In [18]:
# 3. 데이터 표준화
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [19]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
import seaborn as sn
from sklearn.decomposition import PCA
pca = PCA(n_components=21)
X_pca = pca.fit_transform(X_scaled)

In [20]:
# PCA 후 특성 수 확인
num_features = X_pca.shape[1]  # PCA 후의 특성 수
num_features

21

In [22]:
# 원-핫 인코딩 (결과가 False,True로 나올 경우가 있음)
y_encoded = pd.get_dummies(y).values

In [23]:
from sklearn.model_selection import train_test_split
X_train,X_test, y_train,y_test = train_test_split(X_pca,y_encoded,test_size=0.2,random_state=0)

In [26]:
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout

# CNN 모델 구성
model = Sequential()

# 타임스텝=window_size, 특성=X_train.shape[2] (특성 수)
model.add(Conv1D(filters=64, kernel_size=2, activation='relu', input_shape=(window_size, X_train.shape[2])))
model.add(MaxPooling1D(pool_size=1))  # MaxPooling 레이어 추가
model.add(Dropout(0.2))  # Dropout 레이어 추가

# 두 번째 Conv1D 레이어
model.add(Conv1D(filters=32, kernel_size=2, activation='relu'))
model.add(MaxPooling1D(pool_size=1))  # MaxPooling 레이어 추가
model.add(Dropout(0.2))  # Dropout 레이어 추가

model.add(Flatten())  # Flatten 레이어 추가하여 출력형태를 변환
model.add(Dense(64, activation='relu'))  # Fully Connected Layer
model.add(Dropout(0.5))  # Dropout 레이어 추가

# 출력층: 단일 연속 값을 예측
model.add(Dense(1))  # 회귀 문제이므로 activation 함수 없음

# 모델 컴파일
model.compile(optimizer='adam', loss='mse', metrics=['mae'])  # 회귀에 적합한 손실 함수와 메트릭

NameError: name 'window_size' is not defined

In [25]:


model_history=model.fit(x=X_train, y=y_train, epochs=30, batch_size=32,validation_data= (X_test,y_test))
y_pred = model.predict(X_test)

y_test_class = np.argmax(y_test,axis=1)
y_pred_class = np.argmax(y_pred,axis=1)

Epoch 1/30


ValueError: Arguments `target` and `output` must have the same shape. Received: target.shape=(None, 11602), output.shape=(None, 1)